In [ ]:
import threading
import time
import multiprocessing as mp
import sys

In [ ]:
class UpDown():
    def __init__(self, initial):
        self.lock = threading.Condition() # *
        self.value = initial

    def up(self):
        with self.lock:
            self.value += 1
            self.lock.notify() # *

    def down(self):
        with self.lock:
            while self.value == 0:
                self.lock.wait() # *
            self.value -= 1
            
class ChopStick():
    def __init__(self, number):
        self.number = number  + 1        
        self.user = -1      
        self.lock = threading.Condition()
        self.taken = False

    def take(self, user):         # used for synchronization
        with self.lock:
            while self.taken == True:
                self.lock.wait()
            self.user = user
            self.taken = True
            sys.stdout.write("Filósofo[%s] tomó el palillo[%s]\n" % (user, self.number))
            self.lock.notifyAll()

    def drop(self, user):         # used for synchronization
        with self.lock:
            while self.taken == False:
                self.lock.wait()
            self.user = -1
            self.taken = False
            sys.stdout.write("Filósofo[%s] dejó el palillo[%s]\n" % (user, self.number))
            self.lock.notifyAll()

class Philosopher (threading.Thread):
    def __init__(self, number, left, right, semaphore):
        threading.Thread.__init__(self)
        self.number = number + 1           
        self.left = left
        self.right = right
        self.semaphore = semaphore
    
    def run(self):
        for i in range(1):
            sys.stdout.write("Filósofo[%s] PENSANDO\n" % (self.number))
            time.sleep(1)
            self.semaphore.down()              
            time.sleep(1)  
            self.left.take(self.number)    
            time.sleep(1)     
            self.right.take(self.number)    
            time.sleep(1)                
            sys.stdout.write("Filósofo[%s] COMIENDO\n" % (self.number))
            self.right.drop(self.number)    
            time.sleep(1)
            self.left.drop(self.number)     
            time.sleep(1)
            self.semaphore.up()             
            #time.sleep(1)
        sys.stdout.write("---------------Filósofo[%s] terminó de COMER--------------\n" % self.number)

def dinner():
    n = int(input('Dime cuántos Filósofos : '))
    c = [ChopStick(i) for i in range(n)]
    semaphore = UpDown(n-1)
    sys.stdout.write("\n")
    p = [Philosopher(i, c[i], c[(i+1)%n], semaphore) for i in range(n)]

    for i in range(n):
        p[i].start()
        
    for j in range(n):
        p[i].join()
    
dinner()

Dime cuántos Filósofos : 4

Filósofo[1] PENSANDO
Filósofo[2] PENSANDO
Filósofo[3] PENSANDO
Filósofo[4] PENSANDO
Filósofo[2] tomó el palillo[2]
Filósofo[1] tomó el palillo[1]
Filósofo[3] tomó el palillo[3]
Filósofo[3] tomó el palillo[4]
Filósofo[3] COMIENDO
Filósofo[3] dejó el palillo[4]
Filósofo[3] dejó el palillo[3]
Filósofo[2] tomó el palillo[3]
Filósofo[2] COMIENDO
Filósofo[2] dejó el palillo[3]
---------------Filósofo[3] terminó de COMER--------------
Filósofo[2] dejó el palillo[2]
Filósofo[1] tomó el palillo[2]
Filósofo[4] tomó el palillo[4]
---------------Filósofo[2] terminó de COMER--------------
Filósofo[1] COMIENDO
Filósofo[1] dejó el palillo[2]
Filósofo[1] dejó el palillo[1]
Filósofo[4] tomó el palillo[1]
---------------Filósofo[1] terminó de COMER--------------
Filósofo[4] COMIENDO
Filósofo[4] dejó el palillo[1]
Filósofo[4] dejó el palillo[4]
---------------Filósofo[4] terminó de COMER--------------
